In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from google.cloud import bigquery

In [4]:
def estimate_bigquery_query_cost(bq_client, query):
    
    job_config = bigquery.QueryJobConfig()
    job_config.dry_run = True
    job_config.use_query_cache = False
    query_job = bq_client.query(

        (
           query
        ),
        job_config=job_config,
    )
    
    cost_euros = (query_job.total_bytes_processed / 1024 ** 4) * 6

    print(f"{query_job.total_bytes_processed} bytes will be processed , cost ~{cost_euros}$")

In [5]:
bigquery_client= bigquery.Client(project="ingka-energy-analytics-dev") # ingka-energy-solar-dev

In [6]:
# Download query results.
query_string = """
    SELECT 
        internal_search_type
        , count(1) as count_cat
    FROM `ingka-web-analytics-prod.web_data_v2.hits_events_and_pages` 
    WHERE 
        date_hit = '2022-07-02' 
        AND internal_search_type is not Null
    GROUP BY internal_search_type
    ORDER BY count_cat DESC
    LIMIT 1000
"""

estimate_bigquery_query_cost(bigquery_client, query_string) 

2510418036 bytes will be processed , cost ~0.013699271417863201$


In [7]:
# df = (
#     bigquery_client.query(query_string)
#     .result()
#     .to_dataframe(
#         # Optionally, explicitly request to use the BigQuery Storage API. As of
#         # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
#         # API is used by default.
#         create_bqstorage_client=True,
#     )
# )

# df

In [12]:
# Get SQL string filter condition
product_string_queries = [
    'solar','sol', 'sunpower', 'svea', 'solstroma', 'solpaneler',
]
string_exp = ' OR '.join([f'"{exp}" in UNNEST(internal_search_terms)' for exp in product_string_queries])
string_exp

'"solar" in UNNEST(internal_search_terms) OR "sol" in UNNEST(internal_search_terms) OR "sunpower" in UNNEST(internal_search_terms) OR "svea" in UNNEST(internal_search_terms) OR "solstroma" in UNNEST(internal_search_terms) OR "solpaneler" in UNNEST(internal_search_terms)'

In [ ]:
# Download query results.
query_string = """
SELECT *
FROM (
    SELECT 
        date_hit
        , visitor_id
        , session_id
        , website_market_short
        , website_language_short
        , ARRAY_AGG(DISTINCT LOWER(internal_search_term)) as internal_search_terms
    FROM `ingka-web-analytics-prod.web_data_v2.hits_events_and_pages` 
    WHERE 
        date_hit >= '2022-07-02' 
        AND website_market_short in ('us','se')
        AND event_category like '%search%'
        AND internal_search_type = 'hard_search'
        AND internal_search_term is not Null
    GROUP BY
        date_hit
        , visitor_id
        , session_id
        , website_market_short
        , website_language_short
    ORDER BY visitor_id
) searches
WHERE 
    {exp}
""".format(exp=string_exp)

estimate_bigquery_query_cost(bigquery_client, query_string) 

In [17]:
# Write to table
table_id="ingka-energy-analytics-dev.ces_da_playground.searches_temp"

job_config = bigquery.QueryJobConfig(
    allow_large_results=True, destination=table_id, use_legacy_sql=False,
    write_disposition = "WRITE_TRUNCATE"
)


bigquery_client.query(
    query_string, 
    job_config=job_config
).result()

In [15]:
job_config = bigquery.QueryJobConfig(
    use_legacy_sql=False
)

df = (
    bigquery_client.query(query_string, job_config)
    .result()
    .to_dataframe(
        # Optionally, explicitly request to use the BigQuery Storage API. As of
        # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
        # API is used by default.
        create_bqstorage_client=True,
    )
)

df

,date_hit,visitor_id,session_id,website_market_short,website_language_short,internal_search_terms
0,2022-07-02,3301425720097539846,33014257200975398461656784646,se,sv,[sol]
1,2022-07-02,5068006246376821932,50680062463768219321656771756,se,sv,"[solsäng äpplarö, utemöbel, solsäng, sol, äppl..."
2,2022-07-02,6204425850977255515,62044258509772555151656750170,se,sv,[sol]
3,2022-07-02,6925438576644375683,69254385766443756831656787898,se,sv,"[sol, solstol]"
4,2022-07-02,7148360895570161200,71483608955701612001656794672,se,sv,"[sun and moon, sol]"
